# <center>EDA for Yahoo Finance Webscrape</center>

In [1]:
import requests
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import numpy
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.0f}'.format

chrome_options = Options()
chrome_options.add_argument('--headless')

In [2]:
is_link = 'https://finance.yahoo.com/quote/AAPL?p=AAPL'

chrome = webdriver.Chrome('/usr/bin/chromedriver', options=chrome_options)
chrome.get(is_link)
button = chrome.find_element_by_name('agree')
chrome.execute_script('arguments[0].click()', button)

html = chrome.execute_script('return document.body.innerHTML;')

soup = BeautifulSoup(html, 'lxml')

In [3]:
close_price = [entry.text for entry in soup.find_all('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})]

In [4]:
print(close_price[0])

123.00


In [8]:
financial_page = 'https://finance.yahoo.com/quote/AAPL/financials?p=AAPL'
chrome.get(financial_page)
html_financial = chrome.execute_script('return document.body.innerHTML;')

soup_financial = BeautifulSoup(html_financial, 'lxml')
print(soup_financial)

<html><body><div class="Ff($yahooSansFinanceFont)!" id="app"><div class="" data-react-checksum="1811399454" data-reactid="1" data-reactroot=""><div data-reactid="2"><div class="render-target-active render-target-default Pos(a) W(100%)" data-reactid="3" id="render-target-default"><div class="Bgc($bg-body) Mih(100%) W(100%) Bgc($layoutBgColor)! finance US" data-reactid="4" style="margin-top:135px;"><div class="YDC-Header Z(10) End(0) Start(0) T(0) H(0) Panel-open_Translate3d(0,-19px,0) hasScrolled_Translate3d(0,-19px,0) Translate3d(0,0,0) Pos(f) Trsp(a) Trsdu(300ms)" data-reactid="5"><div class="YDC-UH Bgi($uhGrayGradient) D(n)--print Pos(r) Z(1)" data-reactid="6" id="YDC-UH"><div class="YDC-UH Miw(1007px) Maw(1260px) tablet_Miw(600px)--noRightRail Bxz(bb) Bdstartc(t) Bdstartw(20px) Bdendc(t) Bdends(s) Bdendw(20px) Bdstarts(s) Mx(a) tablet_Bd(n)" data-reactid="7" id="YDC-UH-Stack"><div data-reactid="8" id="YDC-UH-Stack-Composite"><div data-reactid="9"><div data-locator="subtree-root" id=

In [9]:
features = soup_financial.find_all('div', class_='D(tbr)')
print(features)

[<div class="D(tbr) C($primaryColor)" data-reactid="37"><div class="D(ib) Fw(b) Ta(start) Px(15px)--mv2 Px(10px) W(247px)--mv2 W(222px) Bxz(bb) Bdendw(1px) Bdstartw(1px) Bdbw(1px) Bdends(s) Bdstarts(s) Bdbs(s) Bdc($seperatorColor) Py(6px) Pos(st) Start(0) Bgc($lv2BgColor)" data-reactid="38"><span data-reactid="39">Breakdown</span><div class="W(3px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n) Pend(5px)" data-reactid="40"></div></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(ib) Fw(b) Tt(u) Bgc($lv1BgColor)" data-reactid="41"><span data-reactid="42">ttm</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(ib) Fw(b)" data-reactid="43"><span data-reactid="44">9/30/2020</span></div><div class="Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(ib) Fw(b) Bgc($lv1BgColor)" data-reactid="45"><span data-reactid="46">9/30/2019</span></div><div clas

In [10]:
headers = []
temp_list = []
label_list = []
final = []
index = 0
#create headers
for item in features[0].find_all('div', class_='D(ib)'):
    headers.append(item.text)
#statement contents
while index <= len(features)-1:
    #filter for each line of the statement
    temp = features[index].find_all('div', class_='D(tbc)')
    for line in temp:
        #each item adding to a temporary list
        temp_list.append(line.text)
    #temp_list added to final list
    final.append(temp_list)
    #clear temp_list
    temp_list = []
    index+=1
df = pd.DataFrame(final[1:])
df.columns = headers

In [12]:
df

,Breakdown,ttm,9/30/2020,9/30/2019,9/30/2018,9/30/2017
0,Total Revenue,"294,135,000","274,515,000","260,174,000","265,595,000","229,234,000"
1,Cost of Revenue,"180,068,000","169,559,000","161,782,000","163,756,000","141,048,000"
2,Gross Profit,"114,067,000","104,956,000","98,392,000","101,839,000","88,186,000"
3,Operating Expense,"39,814,000","38,668,000","34,462,000","30,941,000","26,842,000"
4,Operating Income,"74,253,000","66,288,000","63,930,000","70,898,000","61,344,000"
5,Net Non Operating Interest Income Expense,"739,000","890,000","1,385,000","2,446,000","2,878,000"
6,Other Income Expense,"-240,000","-87,000","422,000","-441,000","-133,000"
7,Pretax Income,"74,752,000","67,091,000","65,737,000","72,903,000","64,089,000"
8,Tax Provision,"10,822,000","9,680,000","10,481,000","13,372,000","15,738,000"
9,Net Income Common Stockholders,"63,930,000","57,411,000","55,256,000","59,531,000","48,351,000"


In [13]:
def convert_to_numeric(column):
    first_col = [i.replace(',','') for i in column]
    second_col = [i.replace('-','') for i in first_col]
    final_col = pd.to_numeric(second_col)
    
    return final_col

In [14]:
for column in headers[1:]:
    df[column] = convert_to_numeric(df[column])

final_df = df.fillna('-')

In [15]:
df

,Breakdown,ttm,9/30/2020,9/30/2019,9/30/2018,9/30/2017
0,Total Revenue,294135000,274515000,260174000,265595000,229234000
1,Cost of Revenue,180068000,169559000,161782000,163756000,141048000
2,Gross Profit,114067000,104956000,98392000,101839000,88186000
3,Operating Expense,39814000,38668000,34462000,30941000,26842000
4,Operating Income,74253000,66288000,63930000,70898000,61344000
5,Net Non Operating Interest Income Expense,739000,890000,1385000,2446000,2878000
6,Other Income Expense,240000,87000,422000,441000,133000
7,Pretax Income,74752000,67091000,65737000,72903000,64089000
8,Tax Provision,10822000,9680000,10481000,13372000,15738000
9,Net Income Common Stockholders,63930000,57411000,55256000,59531000,48351000
